# AVI 3 Migration Sample

This is for first steps... of AVI by using Jupyter

https://avinetworks.com/docs/latest/api-guide/overview.html 


https://github.com/avinetworks/sdk/blob/master/python/avi/sdk/README.md

[1.- Initial Login](#1--initial-login)
[2.- Read Operations](#import-libraries)

## Initial Login

In [107]:
from avi.sdk.avi_api import ApiSession
import datetime, time
from requests.packages import urllib3
urllib3.disable_warnings()
import json
import pandas as pd
from IPython.display import display

# Import environment variables with controller information and credentials
from envs.controller_info import session_params as session_env

# Establish a first session with AVI Controller
api = ApiSession(
    controller_ip=session_env['controller_ip'],
    username=session_env['controller_username'],
    password=session_env['controller_password'],
    tenant=session_env['tenant'],
    api_version=session_env['api_version']
    )
# Update headers and api version imported from demo env file with controller version (ensure actual API Version is uses in subsequent requests
session_env['headers']['X-Avi-Version'] = api.remote_api_version['Version']
session_env['api_version'] = api.remote_api_version['Version']

# Create a new session with received AVI API Version
api = ApiSession(
    controller_ip=session_env['controller_ip'],
    username=session_env['controller_username'],
    password=session_env['controller_password'],
    tenant=session_env['tenant'],
    api_version=session_env['api_version']
    )
# Display Session ID to Verify AVI Controller Session Establishment
print('Successful connection to ' + session_env['name'] + '. Session ID:' + api.session_id)

Successful connection to avicontroller. Session ID:9ax857x8oxlapvtxsmt9xrrl7dr4qgjf


## 2.- Read Operations

### 2.1 Basic Operations via JSON

Import some required Libraries first

In [ ]:
# Extract information from MAIN AND MIGRATED SEGROUP
# MAIN --> Target SE Group where consolditated VS will be placed
# MIGRATED --> SE Group to consolidate into MAIN SE Group after configuration extraction

The first step is to Gather SE Group Information
- Service Engine Group Configuration
- Service Engine Configuration
- Virtual Services Configuration
- Network Services Configuration (if exists)

In [151]:
# Set consolidated name
source_segroup = "MAD-SEG002"
target_segroup = "MAD-SEG001"

In [103]:
# GET Service Engine Group Configuration


print("\033[1mExtracting SOURCE Service Engine Group Information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")
print()

# Define GET request parameters
url_path = "serviceenginegroup"
query = {
   "skip_default": "true",
   "name": source_segroup
}
# Send GET Request
resp = api.get(url_path, params=query)
print ("Request sent to URL: " + resp.url)

# Control Response Status Code
if resp.status_code in range(200, 299):
   print(resp)
   print(resp.reason)
    
   # Convert response JSON into Python Dictionary
   resp_data = json.loads(resp.text)

   # Extract Result
   resp_data = resp_data["results"]
   resp_names = [resp_name["name"] for resp_name in resp_data]
   print()
   print("The following " + url_path + " names has been found:")
   print(resp_names)
   print()
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)

# Save Result
migrating_segroup_data = resp_data[0]

# Extract related virtual services
# GET Virtual Services Related Configuration
print("\033[1mExtracting SOURCE Virtual Services Information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")
print()

seg_data = migrating_segroup_data
# Define GET request parameters
se_group_uuid=seg_data["uuid"]
url_path = "virtualservice"
query = {
   "skip_default": "true",
   "refers_to": f"serviceenginegroup:{se_group_uuid}"
}

# Send GET Request
resp = api.get(url_path, params=query)
print ("Request sent to URL: " + resp.url)

# Control Response Status Code
if resp.status_code in range(200, 299):
   print(resp)
   print(resp.reason)
    
   # Convert response JSON into Python Dictionary
   resp_data = json.loads(resp.text)

   # Extract Result
   resp_data = resp_data["results"]
   resp_names = [resp_name["name"] for resp_name in resp_data]
   print()
   print("The following " + url_path + " names has been found:")
   print(resp_names)
   print()
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)

# Save Result
vs_data = resp_data

# Extract related Service Engine
print("\033[1mExtracting SOURCE Services Engines information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")
print()

# GET Service Engine Related Configuration

# Define GET request parameters
se_group_uuid=seg_data["uuid"]
url_path = "serviceengine"
query = {
   "skip_default": "true",
   "refers_to": f"serviceenginegroup:{se_group_uuid}"
}

# Send GET Request
resp = api.get(url_path, params=query)
print ("Request sent to URL: " + resp.url)

# Control Response Status Code
if resp.status_code in range(200, 299):
   print(resp)
   print(resp.reason)
    
   # Convert response JSON into Python Dictionary
   resp_data = json.loads(resp.text)

   # Extract Result
   resp_data = resp_data["results"]
   resp_names = [resp_name["name"] for resp_name in resp_data]
   print()
   print("The following " + url_path + " names has been found:")
   print(resp_names)
   print()
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)

# Save Result
migrating_se_data = resp_data

Extracting SOURCE Service Engine Group Information
--------------------------------------------------

Request sent to URL: https://192.168.1.15/api/serviceenginegroup?skip_default=true&name=MAD-SEG002
<Response [200]>
OK

The following serviceenginegroup names has been found:
['MAD-SEG002']

Extracting SOURCE Virtual Services Information
--------------------------------------------------

Request sent to URL: https://192.168.1.15/api/virtualservice?skip_default=true&refers_to=serviceenginegroup%3Aserviceenginegroup-932a4c8c-ee69-4970-94dc-e3d9fc42032f
<Response [200]>
OK

The following virtualservice names has been found:
['vs-example-mad-002-01', 'vs-example-mad-002-02']

Extracting SOURCE Services Engines information
--------------------------------------------------

Request sent to URL: https://192.168.1.15/api/serviceengine?skip_default=true&refers_to=serviceenginegroup%3Aserviceenginegroup-932a4c8c-ee69-4970-94dc-e3d9fc42032f
<Response [200]>
OK

The following serviceengine name

## Exploring Source Information

In [128]:
# Exploring Interface Information of SOURCE SE

for i in range(len(migrating_se_data)):
    print("\033[1mShowing information of the SOURCE SE "+str(i+1)+" \033[0m")
    print("\033[1m--------------------------------------------------\033[0m")
    se_data_vnics = migrating_se_data[i]["data_vnics"]
    se_df = pd.DataFrame(se_data_vnics)
    display(se_df)
    time.sleep(1/4)


Showing information of the SOURCE SE 1 
--------------------------------------------------


,adapter,connected,dhcp_enabled,if_name,ip6_autocfg_enabled,linux_name,mac_address,port_uuid,vrf_ref,enabled,vnic_networks
0,Unknown,True,False,eth3,False,eth3,00:0c:29:a7:83:62,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN,NaN
1,Unknown,True,False,eth4,False,eth6,00:0c:29:a7:83:6c,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN,NaN
2,Unknown,True,False,eth6,False,eth8,00:0c:29:a7:83:76,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN,NaN
3,Unknown,True,False,eth8,False,eth1,00:0c:29:a7:83:80,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN,NaN
4,Unknown,True,False,eth1,False,eth4,00:0c:29:a7:83:8a,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,False,"[{'ip': {'ip_addr': {'addr': '192.168.4.222', ..."
5,Unknown,True,False,eth2,False,eth7,00:0c:29:a7:83:94,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN,NaN
6,Unknown,True,False,eth5,False,eth9,00:0c:29:a7:83:9e,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN,NaN
7,Unknown,True,False,eth7,False,eth2,00:0c:29:a7:83:a8,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN,NaN
8,Unknown,True,False,eth9,False,eth5,00:0c:29:a7:83:b2,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN,NaN


Showing information of the SOURCE SE 2 
--------------------------------------------------


,adapter,connected,dhcp_enabled,if_name,ip6_autocfg_enabled,linux_name,mac_address,port_uuid,vrf_ref,vnic_networks
0,Unknown,True,False,eth3,False,eth3,00:0c:29:a7:61:16,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
1,Unknown,True,False,eth4,False,eth6,00:0c:29:a7:61:20,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
2,Unknown,True,False,eth6,False,eth8,00:0c:29:a7:61:2a,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
3,Unknown,True,False,eth8,False,eth1,00:0c:29:a7:61:34,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
4,Unknown,True,False,eth1,False,eth4,00:0c:29:a7:61:3e,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,"[{'ip': {'ip_addr': {'addr': '192.168.4.223', ..."
5,Unknown,True,False,eth2,False,eth7,00:0c:29:a7:61:48,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
6,Unknown,True,False,eth5,False,eth9,00:0c:29:a7:61:52,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
7,Unknown,True,False,eth7,False,eth2,00:0c:29:a7:61:5c,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
8,Unknown,True,False,eth9,False,eth5,00:0c:29:a7:61:66,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN


### Select Interface to be migrated (ensure you are picking if_name and not linux_name column)


In [ ]:
if_name_to_migrate = "eth1"

In [147]:
# Extract Interface Information from first SE
output_config=[]
for i in range(len(migrating_se_data)):
    config=[]
    print("\033[1mShowing information of the SOURCE SE "+str(i+1)+" interface to migrate "+if_name_to_migrate+" \033[0m")
    print("\033[1m--------------------------------------------------\033[0m")
    se_data_vnics = migrating_se_data[i]["data_vnics"]
    se_data_vnic_to_migrate = [ adapter for adapter in se_data_vnics if adapter.get("if_name") == if_name_to_migrate]
    se_data_vnic_to_migrate_df = pd.DataFrame(se_data_vnic_to_migrate)
    time.sleep(1/4)
    display(se_data_vnic_to_migrate_df)
    se_vnic_to_migrate_config = se_data_vnic_to_migrate[0]["vnic_networks"][0]["ip"]
    se_vnic_to_migrate_config_df = pd.DataFrame(se_vnic_to_migrate_config)
    display(se_vnic_to_migrate_config_df)
    config={"ip_addr":se_data_vnic_to_migrate[0]["vnic_networks"][0]["ip"]["ip_addr"]["addr"],
            "mask": se_data_vnic_to_migrate[0]["vnic_networks"][0]["ip"]["mask"],
            "mac_address": se_data_vnic_to_migrate[0]["mac_address"]}
    output_config.append(config)


Showing information of the SOURCE SE 1 interface to migrate eth1 
--------------------------------------------------


,adapter,connected,dhcp_enabled,enabled,if_name,ip6_autocfg_enabled,linux_name,mac_address,port_uuid,vnic_networks,vrf_ref
0,Unknown,True,False,False,eth1,False,eth4,00:0c:29:a7:83:8a,Unknown,"[{'ip': {'ip_addr': {'addr': '192.168.4.222', ...",https://192.168.1.15/api/vrfcontext/vrfcontext...


,ip_addr,mask
addr,192.168.4.222,24
type,V4,24


Showing information of the SOURCE SE 2 interface to migrate eth1 
--------------------------------------------------


,adapter,connected,dhcp_enabled,if_name,ip6_autocfg_enabled,linux_name,mac_address,port_uuid,vnic_networks,vrf_ref
0,Unknown,True,False,eth1,False,eth4,00:0c:29:a7:61:3e,Unknown,"[{'ip': {'ip_addr': {'addr': '192.168.4.223', ...",https://192.168.1.15/api/vrfcontext/vrfcontext...


,ip_addr,mask
addr,192.168.4.223,24
type,V4,24


In [150]:
print ("The following interfaces has been selected for migration")
print()
for i in range(len(output_config)):
    print("For the Service Engine "+str(i+1)+" with name \033[1m" + migrating_se_data[i]["name"]+"\033[0m")
    print("The interface \033[1m" + if_name_to_migrate + "\033[0m has been selected with following configuration")
    print("   IP Address \033[1m" + output_config[i]["ip_addr"] +"\033[0m")
    print("   Mask Length \033[1m" + str(output_config[i]["mask"]) +"\033[0m")
    print("   MAC-ADDRESS \033[1m" + output_config[i]["mac_address"] +"\033[0m")
    print()

The following interfaces has been selected for migration

For the Service Engine 1 with name fd38:9bda:673d:0:20c:29ff:fea7:8358
The interface eth1 has been selected with following configuration
   IP Address 192.168.4.222
   Mask Length 24
   MAC-ADDRESS 00:0c:29:a7:83:8a

For the Service Engine 2 with name fd38:9bda:673d:0:20c:29ff:fea7:610c
The interface eth1 has been selected with following configuration
   IP Address 192.168.4.223
   Mask Length 24
   MAC-ADDRESS 00:0c:29:a7:61:3e



### TARGET SERVICE ENGINE GROUP

In [154]:
# GET Service Engine Group Configuration

# Define GET request parameters
# Extract related Service Engine
print("\033[1mExtracting TARGET Services Engine Group information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")
print()

url_path = "serviceenginegroup"
query = {
   "skip_default": "true",
   "name": target_segroup
}
# Send GET Request
resp = api.get(url_path, params=query)
print ("Request sent to URL: " + resp.url)

# Control Response Status Code
if resp.status_code in range(200, 299):
   print(resp)
   print(resp.reason)
    
   # Convert response JSON into Python Dictionary
   resp_data = json.loads(resp.text)

   # Extract Result
   resp_data = resp_data["results"]
   resp_names = [resp_name["name"] for resp_name in resp_data]
   print()
   print("The following " + url_path + " names has been found:")
   print(resp_names)
   print()
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)

# Save Result
target_segroup_data = resp_data[0]

# Extract related Service Engine

# GET Service Engine Related Configuration
print("\033[1mExtracting TARGET Services Engine Group information\033[0m")
print("\033[1m--------------------------------------------------\033[0m")
print()

segroup_data = target_segroup_data
# Define GET request parameters
se_group_uuid=segroup_data["uuid"]
url_path = "serviceengine"
query = {
   "skip_default": "true",
   "refers_to": f"serviceenginegroup:{se_group_uuid}"
}

# Send GET Request
resp = api.get(url_path, params=query)
print ("Request sent to URL: " + resp.url)

# Control Response Status Code
if resp.status_code in range(200, 299):
   print(resp)
   print(resp.reason)
    
   # Convert response JSON into Python Dictionary
   resp_data = json.loads(resp.text)

   # Extract Result
   resp_data = resp_data["results"]
   resp_names = [resp_name["name"] for resp_name in resp_data]
   print()
   print("The following " + url_path + " names has been found:")
   print(resp_names)
   print()
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)

# Save Result
target_se_data = resp_data

Extracting TARGET Services Engine Group information
--------------------------------------------------

Request sent to URL: https://192.168.1.15/api/serviceenginegroup?skip_default=true&name=MAD-SEG001
<Response [200]>
OK

The following serviceenginegroup names has been found:
['MAD-SEG001']

Extracting TARGET Services Engine Group information
--------------------------------------------------

Request sent to URL: https://192.168.1.15/api/serviceengine?skip_default=true&refers_to=serviceenginegroup%3Aserviceenginegroup-e5b13687-efbb-4417-9302-b1d37c367c8b
<Response [200]>
OK

The following serviceengine names has been found:
['fd38:9bda:673d:0:20c:29ff:fec4:a7af', 'fd38:9bda:673d:0:20c:29ff:feba:47eb']



In [156]:
# Exploring Interface Information of SOURCE SE

for i in range(len(target_se_data)):
    print("\033[1mShowing information of the TARGET SE "+str(i+1)+" \033[0m")
    print("\033[1m--------------------------------------------------\033[0m")
    se_data_vnics = target_se_data[i]["data_vnics"]
    se_df = pd.DataFrame(se_data_vnics)
    display(se_df)
    time.sleep(1/4)

Showing information of the TARGET SE 1 
--------------------------------------------------


,adapter,connected,dhcp_enabled,if_name,ip6_autocfg_enabled,linux_name,mac_address,port_uuid,vrf_ref,vnic_networks
0,Unknown,True,False,eth9,False,eth5,00:0c:29:c4:a7:09,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
1,Unknown,True,False,eth3,False,eth3,00:0c:29:c4:a7:b9,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
2,Unknown,True,False,eth4,False,eth6,00:0c:29:c4:a7:c3,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
3,Unknown,True,False,eth6,False,eth8,00:0c:29:c4:a7:cd,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
4,Unknown,True,False,eth8,False,eth1,00:0c:29:c4:a7:d7,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
5,Unknown,True,False,eth1,False,eth4,00:0c:29:c4:a7:e1,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,"[{'ip': {'ip_addr': {'addr': '192.168.1.40', '..."
6,Unknown,True,False,eth2,False,eth7,00:0c:29:c4:a7:eb,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
7,Unknown,True,False,eth5,False,eth9,00:0c:29:c4:a7:f5,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
8,Unknown,True,False,eth7,False,eth2,00:0c:29:c4:a7:ff,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN


Showing information of the TARGET SE 2 
--------------------------------------------------


,adapter,connected,dhcp_enabled,if_name,ip6_autocfg_enabled,linux_name,mac_address,port_uuid,vrf_ref,vnic_networks
0,Unknown,True,False,eth6,False,eth8,00:0c:29:ba:47:09,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
1,Unknown,True,False,eth8,False,eth1,00:0c:29:ba:47:13,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
2,Unknown,True,False,eth1,False,eth4,00:0c:29:ba:47:1d,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,"[{'ip': {'ip_addr': {'addr': '192.168.1.41', '..."
3,Unknown,True,False,eth2,False,eth7,00:0c:29:ba:47:27,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
4,Unknown,True,False,eth5,False,eth9,00:0c:29:ba:47:31,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
5,Unknown,True,False,eth7,False,eth2,00:0c:29:ba:47:3b,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
6,Unknown,True,False,eth9,False,eth5,00:0c:29:ba:47:45,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
7,Unknown,True,False,eth3,False,eth3,00:0c:29:ba:47:f5,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN
8,Unknown,True,False,eth4,False,eth6,00:0c:29:ba:47:ff,Unknown,https://192.168.1.15/api/vrfcontext/vrfcontext...,NaN


### Select one FREE interface that will be used as target


In [157]:
if_name_target = "eth2"

In [62]:
# Assuming IP Address will be REUSED!!!
#  1.- Migrate VS (change SEGroup)
#  2.- Migrate SEs interfaces
#  3.- Migrate Network Services (if exists)
# Disable MIGRATING SE INTERFACES TO AVOID ITF CLASHING

## Migrate Service Engine Interfaces

In [161]:
# Display gathered information prior to execute next cell


print ("The following interfaces has been selected for migration")
print()
for i in range(len(output_config)):
    print("For the SOURCE Service Engine "+str(i+1)+" with name \033[1m" + migrating_se_data[i]["name"]+"\033[0m")
    print("The interface \033[1m" + if_name_to_migrate + "\033[0m has been selected with following configuration")
    print("   IP Address \033[1m" + output_config[i]["ip_addr"] +"\033[0m")
    print("   Mask Length \033[1m" + str(output_config[i]["mask"]) +"\033[0m")
    print()
print("---------------------")
for i in range(len(target_se_data)):
    print("For the TARGET Service Engine "+str(i+1)+" with name \033[1m" + target_se_data[i]["name"]+"\033[0m")
    print("The interface \033[1m" + if_name_target + "\033[0m will be configured with above information.")
    print("   IP Address \033[1m" + output_config[i]["ip_addr"] +"\033[0m")
    print("   Mask Length \033[1m" + str(output_config[i]["mask"]) +"\033[0m")
    print()
print("---------------------")
print("WARNING!!! Before proceeding migrated interfaces in the migrating SE's will be disabled")

The following interfaces has been selected for migration

For the SOURCE Service Engine 1 with name fd38:9bda:673d:0:20c:29ff:fea7:8358
The interface eth1 has been selected with following configuration
   IP Address 192.168.4.222
   Mask Length 24

For the SOURCE Service Engine 2 with name fd38:9bda:673d:0:20c:29ff:fea7:610c
The interface eth1 has been selected with following configuration
   IP Address 192.168.4.223
   Mask Length 24

---------------------
For the TARGET Service Engine 1 with name fd38:9bda:673d:0:20c:29ff:fec4:a7af
The interface eth2 will be configured with above information.
   IP Address 192.168.4.222
   Mask Length 24

For the TARGET Service Engine 2 with name fd38:9bda:673d:0:20c:29ff:feba:47eb
The interface eth2 will be configured with above information.
   IP Address 192.168.4.223
   Mask Length 24

---------------------
WARNING!!! Before proceeding migrated interfaces in the migrating SE's will be disabled


In [98]:
# Select one FREE interface that will be used as target
if_name_source = if_name_to_migrate
if_name_target = "eth2"

In [164]:
# Create dictionary list containint pair of migrating-target SEs  
se_pairs =[
     {"migrating_se_uuid": migrating_se_data[0]["uuid"],
      "target_se_uuid": target_se_data[0]["uuid"]
     },
     {"migrating_se_uuid": migrating_se_data[1]["uuid"],
     "target_se_uuid": target_se_data[1]["uuid"]
     }
]
# Loop to migrate 2 se Pairs (migrating to target) 
for i in range(len(se_pairs)):
    migrating_se_uuid = se_pairs[i]["migrating_se_uuid"]
    target_se_uuid = se_pairs[i]["target_se_uuid"]
    print("Migrating pair "+str(i+1))
    print("-------------------------")
    print("Migrating (source) Service Engine UUID \033[1m"+migrating_se_uuid +"\033[0m at SE Group \033[1m"+source_segroup+"\033[0m") 
    print("Receiving (target) Service Engine UUID \033[1m"+target_se_uuid+"\033[0m at SE Group \033[1m"+target_segroup+"\033[0m")
    migrating_se_uuid = se_pairs[i]["migrating_se_uuid"]
    target_se_uuid = se_pairs[i]["target_se_uuid"]

    # Disabling interface to migrate at source SE

    # Extract the source/target SE data with matching uuid
    source_se_json_data = [ se for se in migrating_se_data if se.get("uuid") == migrating_se_uuid]
    target_se_json_data = [ se for se in target_se_data if se.get("uuid") == target_se_uuid]


    # Extract source_interface vnic_networks object
    json_data_migrating_adapter = [ data_vnic for data_vnic in source_se_json_data[0]["data_vnics"] if data_vnic.get("if_name") == if_name_source ]
    json_data_migrating_adapter_vnic_networks = [json_data_migrating_adapter[0]["vnic_networks"][0]]

    # Modifying existing value for selected interface at source SE (i.e Enabled = False for corresponding interface to shutdown )
    # vnic_network list must be removed to avoid SE interface overlapping
    if_name_key = "if_name"
    if_name_value = if_name_source

    # Loop through each dictionary in the list
    for adapter in source_se_json_data[0]["data_vnics"]:
        # Check if the key_to_check matches the value_to_match
        if adapter.get(if_name_key) == if_name_value:
            # Update the value of key_to_modify
            adapter["enabled"] = False
            adapter["vnic_networks"]=[]
    
    # Remove _last_modified key
    source_se_json_data[0].pop("_last_modified", None)

    # Disable migrating interfaces to avoid IP address overlapping
    body = source_se_json_data[0]
    print ("Applying changes to source SE "+str(i+1)+" with uuid "+ migrating_se_uuid)
    print (" - Changing interface "+if_name_source+" to disabled state")
    print (" - Removing IP Address Configuration")

    url_path = "serviceengine/"+migrating_se_uuid
    resp = api.put (url_path, data=json.dumps(body))

    if resp.status_code in range(200, 299):
      print(resp)
      print('- Object '+url_path+' named '+body['name']+ " modified", resp.reason)#, resp.text)
      print()
    else:
      print('Error in modifying '+url_path+' :%s' % resp.text)

    # PUT to configure migrating IP Addresses in target SE
    # Extract IP Address from migrating SE
    # Importing extracted information into target body
    if_name_key = "if_name"
    if_name_value = if_name_target

    # Loop through each dictionary in the list
    for item in target_se_json_data[0]["data_vnics"]:
      # Check if the interface name correspondes to given value
      if item.get(if_name_key) == if_name_value:
        # Update the value of vnic_networks from source SE
        item["vnic_networks"] = json_data_migrating_adapter_vnic_networks
    
    # Remove _last_modified key
    target_se_json_data[0].pop("_last_modified", None)

    body = target_se_json_data[0]
    print ("Applying changes to target SE "+str(i+1)+" with uuid "+ target_se_uuid)
    print (" - Configuring IP Address Configuration")

    url_path = "serviceengine/"+target_se_uuid
    resp = api.put (url_path, data=json.dumps(body))

    if resp.status_code in range(200, 299):
      print(resp)
      print('- Object '+url_path+' named '+body['name']+ " modified", resp.reason)#, resp.text)
      print()
    else:
      print('Error in modifying '+url_path+' :%s' % resp.text)

Migrating pair 1
-------------------------
Migrating (source) Service Engine UUID se-564d63c6-3882-ca52-7be5-b04c9ba78358 at SE Group MAD-SEG002
Receiving (target) Service Engine UUID se-564dcb90-ff93-dc99-bf4a-62907bc4a7af at SE Group MAD-SEG001
Applying changes to source SE 1 with uuid se-564d63c6-3882-ca52-7be5-b04c9ba78358
 - Changing interface eth1 to disabled state
 - Removing IP Address Configuration
<Response [200]>
- Object serviceengine/se-564d63c6-3882-ca52-7be5-b04c9ba78358 named fd38:9bda:673d:0:20c:29ff:fea7:8358 modified OK

Applying changes to target SE 1 with uuid se-564dcb90-ff93-dc99-bf4a-62907bc4a7af
 - Configuring IP Address Configuration
<Response [200]>
- Object serviceengine/se-564dcb90-ff93-dc99-bf4a-62907bc4a7af named fd38:9bda:673d:0:20c:29ff:fec4:a7af modified OK

Migrating pair 2
-------------------------
Migrating (source) Service Engine UUID se-564d0f61-3a27-ff26-8af7-306296a7610c at SE Group MAD-SEG002
Receiving (target) Service Engine UUID se-564d5830-e